In [143]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori as ap, association_rules as ap_rl
import numpy as np
from icecream import ic
import os
import tensorflow as tf
import keras
from keras import layers
from datetime import date, time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [144]:
export = pd.read_csv("export_nodup.csv")
export = export.drop('Unnamed: 0', axis=1)
export.head(20)


,customer_id,account_id,ed_id,event_name,Date,Time,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22,08:45:22,1
1,-784961211,1773350293,19,application_web_view,2023-03-22,13:32:10,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22,13:32:10,3
3,-784961211,1773350293,2,campaign_click,2023-03-22,14:45:22,4
4,-784961211,1773350293,19,application_web_view,2023-07-27,14:57:56,5
5,-784961211,1773350293,19,application_web_view,2023-08-29,16:01:06,6
6,15849251,383997507,4,browse_products,2021-11-04,14:11:15,1
7,15849251,383997507,4,browse_products,2021-11-04,14:11:29,2
8,15849251,383997507,4,browse_products,2021-11-04,14:12:10,3
9,15849251,383997507,4,browse_products,2021-11-04,14:12:21,4


In [145]:
# Sum up Date and Time column 
export["timestamp"] = pd.to_datetime(export['Date'] + ' ' + export['Time'])

# Convert 'Timestamp' column to datetime format
export['timestamp'] = pd.to_datetime(export['timestamp'])

# Sort the DataFrame by IDs and 'Timestamp'
export = export.sort_values(by=['customer_id', 'account_id', 'timestamp'])

# Calculate the time difference for each row in the same ID
export['Time_Difference'] = export.groupby(by=['customer_id','account_id'])['timestamp'].diff()

# Print the DataFrame with time differences
export.head(30)

# Shift up time column and change NaT into 00:00:00
export['Time_Difference'] = export['Time_Difference'].shift(-1)
export['Time_Difference'] = export['Time_Difference'].fillna(pd.Timedelta(seconds=0))

export.head(60)



,customer_id,account_id,ed_id,event_name,Date,Time,journey_steps_until_end,timestamp,Time_Difference
35435016,-2147483541,2059949086,12,application_web_approved,2021-10-05,08:11:26,1,2021-10-05 08:11:26,0 days 00:05:44
35435017,-2147483541,2059949086,5,view_cart,2021-10-05,08:17:10,2,2021-10-05 08:17:10,0 days 00:04:21
35435018,-2147483541,2059949086,4,browse_products,2021-10-05,08:21:31,3,2021-10-05 08:21:31,0 days 00:00:28
35435019,-2147483541,2059949086,11,add_to_cart,2021-10-05,08:21:59,4,2021-10-05 08:21:59,0 days 00:00:18
35435020,-2147483541,2059949086,5,view_cart,2021-10-05,08:22:17,5,2021-10-05 08:22:17,0 days 00:00:14
35435021,-2147483541,2059949086,6,begin_checkout,2021-10-05,08:22:31,6,2021-10-05 08:22:31,0 days 00:31:04
35435022,-2147483541,2059949086,1,promotion_created,2021-10-05,08:53:35,7,2021-10-05 08:53:35,0 days 13:39:43
35435023,-2147483541,2059949086,4,browse_products,2021-10-05,22:33:18,8,2021-10-05 22:33:18,0 days 00:00:13
35435024,-2147483541,2059949086,11,add_to_cart,2021-10-05,22:33:31,9,2021-10-05 22:33:31,0 days 00:00:16
35435025,-2147483541,2059949086,4,browse_products,2021-10-05,22:33:47,10,2021-10-05 22:33:47,0 days 00:00:13


In [146]:
# select cols
export = export[["customer_id","account_id","ed_id","Time_Difference"]]
# select customer/account/ed_id, count each observation
export["count"] = export.groupby(by = ["customer_id","account_id","ed_id"])["ed_id"].transform("count")

export.head(60)



,customer_id,account_id,ed_id,Time_Difference,count
35435016,-2147483541,2059949086,12,0 days 00:05:44,1
35435017,-2147483541,2059949086,5,0 days 00:04:21,5
35435018,-2147483541,2059949086,4,0 days 00:00:28,4
35435019,-2147483541,2059949086,11,0 days 00:00:18,4
35435020,-2147483541,2059949086,5,0 days 00:00:14,5
35435021,-2147483541,2059949086,6,0 days 00:31:04,3
35435022,-2147483541,2059949086,1,0 days 13:39:43,9
35435023,-2147483541,2059949086,4,0 days 00:00:13,4
35435024,-2147483541,2059949086,11,0 days 00:00:16,4
35435025,-2147483541,2059949086,4,0 days 00:00:13,4


In [147]:
export = export.groupby(["customer_id","account_id","ed_id","count"])['Time_Difference'].sum()


In [148]:
export = pd.DataFrame(export)


In [149]:
export = export.reset_index().rename(columns={'index': 'ed_id'})
export.head(20)


,customer_id,account_id,ed_id,count,Time_Difference
0,-2147483541,2059949086,1,9,111 days 03:59:36
1,-2147483541,2059949086,4,4,0 days 00:01:13
2,-2147483541,2059949086,5,5,0 days 00:06:35
3,-2147483541,2059949086,6,3,37 days 22:58:29
4,-2147483541,2059949086,11,4,0 days 00:01:12
5,-2147483541,2059949086,12,1,0 days 00:05:44
6,-2147483541,2059949086,21,3,89 days 19:45:25
7,-2147481037,-1245770865,1,1,0 days 00:49:50
8,-2147481037,-1245770865,2,1,0 days 03:30:54
9,-2147481037,-1245770865,4,1,0 days 00:00:00


In [150]:
export['Time_Difference'] = export['Time_Difference'].dt.total_seconds()


In [151]:
export

,customer_id,account_id,ed_id,count,Time_Difference
0,-2147483541,2059949086,1,9,9604776.0
1,-2147483541,2059949086,4,4,73.0
2,-2147483541,2059949086,5,5,395.0
3,-2147483541,2059949086,6,3,3279509.0
4,-2147483541,2059949086,11,4,72.0
...,...,...,...,...,...
13037640,2147482120,-488077144,19,1,3845.0
13037641,2147483471,1556733004,1,7,17991677.0
13037642,2147483471,1556733004,4,3,2679345.0
13037643,2147483471,1556733004,12,1,288.0


In [234]:
export_piv2 = export.pivot(index = ["customer_id","account_id"], columns="ed_id", values="count").fillna(0)
export_piv2.head(20)


,ed_id,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,26,27,28,29
customer_id,account_id,,,,,,,,,,,,,,,,,,,,,
-2147483541,2059949086,9.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147481037,-1245770865,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147474335,-1929965268,1.0,1.0,1.0,8.0,2.0,1.0,1.0,1.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
-2147474305,1098145658,7.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,0.0,0.0,...,12.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147472167,889210381,1.0,0.0,1.0,9.0,8.0,5.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
-2147466542,-823743386,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147466262,-296951572,1.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147464266,-640969143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2147460916,1957045389,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [235]:
export_piv2 = pd.DataFrame(export_piv2)
export_piv2 = export_piv2.reset_index()
export_piv2 = export_piv2.drop(['customer_id'], axis=1)
export_piv2.head(20)

ed_id,account_id,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,26,27,28,29
0,2059949086,9.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1245770865,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1929965268,1.0,1.0,1.0,8.0,2.0,1.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,1098145658,7.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,0.0,...,12.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,889210381,1.0,0.0,1.0,9.0,8.0,5.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
5,-823743386,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-296951572,1.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-640969143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1957045389,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-1887501660,6.0,0.0,1.0,19.0,2.0,0.0,0.0,0.0,0.0,...,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
column_names = {'ed_id':'ndex','account_id':'account_id','1':'1c','2':'2c','3':'3c','4':'4c','5':'5c','6':'6c','7':'7c','8':'8c','9':'9c','10':'10c','11':'11c','12':'12c','13':'13c','14':'14c','15':'15c','16':'16c','17':'17c','18':'18c','19':'19c','20':'20c','21':'21c','22':'22c','23':'23c','24':'24c','26':'26c','27':'27c','28':'28c','29':'29c'}
export_piv2.columns = ['account_id','1c','2c','3c','4c','5c','6c','7c','8c','9c','10c','11c','12c','13c','14c','15c','16c','17c','18c','19c','20c','21c','22c','23c','24c','26c','27c','28c','29c']



In [237]:
export_piv2.head(20)

,account_id,1c,2c,3c,4c,5c,6c,7c,8c,9c,...,19c,20c,21c,22c,23c,24c,26c,27c,28c,29c
0,2059949086,9.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1245770865,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1929965268,1.0,1.0,1.0,8.0,2.0,1.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,1098145658,7.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,0.0,...,12.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,889210381,1.0,0.0,1.0,9.0,8.0,5.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
5,-823743386,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-296951572,1.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-640969143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1957045389,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-1887501660,6.0,0.0,1.0,19.0,2.0,0.0,0.0,0.0,0.0,...,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [238]:
cond1 = export_piv2['7c'] > 0 
cond2 = export_piv2['18c'] > 0
cond3 = export_piv2['29c'] > 0

#Purchase and Activate=0, Purchase and non-Activate=1, non-Purchase and non-Activate=2, non-Purchase and Activate=3
conditions  = [(cond1 | cond2) & cond3, (cond1 | cond2) & ~cond3, (~cond1 | ~cond2) & ~cond3, (~cond1 | ~cond2) & cond3]
choices     = [0,1,2,3]
    
export_piv2["Condition"] = np.select(conditions, choices, default=np.nan)



In [239]:
export_piv2

,account_id,1c,2c,3c,4c,5c,6c,7c,8c,9c,...,20c,21c,22c,23c,24c,26c,27c,28c,29c,Condition
0,2059949086,9.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,-1245770865,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,-1929965268,1.0,1.0,1.0,8.0,2.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,1098145658,7.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,889210381,1.0,0.0,1.0,9.0,8.0,5.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735762,-148210516,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1735763,-1286764612,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0
1735764,496493954,5.0,1.0,2.0,52.0,11.0,7.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0
1735765,-488077144,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [240]:
sum(export_piv2['Condition']==0)/1735767

0.2063859953553674

In [241]:
sum(export_piv2['Condition']==1)/1735767

0.014999709062333827

In [242]:
sum(export_piv2['Condition']==2)/1735767

0.7450827213560345

In [243]:
sum(export_piv2['Condition']==3)/1735767

0.03353157422626424

In [244]:
(sum(export_piv2['Condition']==1)/1735767) +(sum(export_piv2['Condition']==2)/1735767) +(sum(export_piv2['Condition']==0)/1735767) +(sum(export_piv2['Condition']==3)/1735767)




1.0

# ANN

In [245]:
a = ['7c','18c','29c','26c','23c','20c','17c','16c','14c','10c','9c']
export_piv2 = export_piv2.drop(columns = a)


In [267]:
export_piv2.head()

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,Condition
0,2059949086,9.0,0.0,0.0,4.0,5.0,3.0,0.0,4.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0
1,-1245770865,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,-1929965268,1.0,1.0,1.0,8.0,2.0,1.0,1.0,3.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1098145658,7.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,12.0,4.0,0.0,0.0,0.0,0.0,2.0
4,889210381,1.0,0.0,1.0,9.0,8.0,5.0,1.0,7.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [268]:
X = export_piv2.iloc[:, 0:18].values

In [269]:
X

array([[ 2.05994909e+09,  9.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.24577086e+09,  1.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.92996527e+09,  1.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [ 4.96493954e+08,  5.00000000e+00,  1.00000000e+00, ...,
         3.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.88077144e+08,  0.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.55673300e+09,  7.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [270]:
Y = export_piv2.iloc[:,-1].values
len(Y)


1735767

In [271]:
Y

array([2., 2., 0., ..., 2., 2., 2.])

In [272]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)


y_encoded = to_categorical(Y_train, 4)

In [273]:
X_train_1 = np.array([inner_array[1:] for inner_array in X_train])
X_test_1 = np.array([inner_array[1:] for inner_array in X_test])
print(X_train_1)


[[8. 1. 2. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]


In [314]:
#Initialising ANN
ann = tf.keras.models.Sequential()

#Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(units=9,activation="relu"))

#Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

#Adding Output Layer
ann.add(tf.keras.layers.Dense(units=4,activation="softmax"))

#Compile ANN
ann.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

#Fitting ANN
ann.fit(X_train_1,y_encoded,batch_size=32,epochs = 50)


Epoch 1/50
37970/37970 [==============================] - 41s 1ms/step - loss: 0.2324 - accuracy: 0.9352
Epoch 2/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.2041 - accuracy: 0.9406
Epoch 3/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1997 - accuracy: 0.9411
Epoch 4/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1966 - accuracy: 0.9415
Epoch 5/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1949 - accuracy: 0.9420
Epoch 6/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1941 - accuracy: 0.9424
Epoch 7/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1934 - accuracy: 0.9425
Epoch 8/50
37970/37970 [==============================] - 42s 1ms/step - loss: 0.1926 - accuracy: 0.9427
Epoch 9/50
37970/37970 [==============================] - 40s 1ms/step - loss: 0.1923 - accuracy: 0.9428
Epoch 10/50
37970/37970 [==============================

In [275]:
probabilities = ann.predict(X_test_1)

16273/16273 [==============================] - 9s 525us/step


In [313]:
predictions = np.argmax(ann.predict(X_test_1), axis=-1)
predictions[1:50]

16273/16273 [==============================] - 8s 519us/step


array([0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
       2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2])

In [277]:
sum(Y_test == predictions)/520731

0.9415686794141312

In [278]:
X_test

array([[ 6.93992986e+08,  9.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.57742620e+07,  6.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [-4.32727077e+08,  9.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-3.14170082e+08,  7.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.09067700e+09,  1.00000000e+00,  1.00000000e+00, ...,
         2.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.34457487e+09,  0.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]])

In [291]:
column_names = ['account_id','1c','2c','3c','4c','5c','6c','8c','11c','12c','13c','15c','19c','21c','22c','24c','27c','28c']
df = pd.DataFrame(X_test, columns=column_names)

['7c','18c','29c','26c','23c','20c','17c','16c','14c','10c','9c']
df['predicted cond'] = predictions


In [292]:
df['Y_test'] = Y_test

In [293]:
df

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
0,6.939930e+08,9.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2,2.0
1,-2.577426e+07,6.0,1.0,0.0,38.0,17.0,7.0,1.0,10.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0.0
2,-4.327271e+08,9.0,0.0,1.0,4.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,2,2.0
3,-1.479181e+09,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,2.0
4,-2.099202e+09,1.0,0.0,1.0,5.0,4.0,2.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520726,-1.429413e+09,6.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2,2.0
520727,-3.118370e+08,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0
520728,-3.141701e+08,7.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2,2.0
520729,1.090677e+09,1.0,1.0,1.0,9.0,9.0,8.0,1.0,2.0,1.0,0.0,0.0,9.0,0.0,0.0,2.0,1.0,1.0,0,0.0


In [294]:

df2 = df[df['predicted cond'] == df['Y_test']]


df2

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
0,6.939930e+08,9.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2,2.0
1,-2.577426e+07,6.0,1.0,0.0,38.0,17.0,7.0,1.0,10.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0.0
2,-4.327271e+08,9.0,0.0,1.0,4.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,2,2.0
3,-1.479181e+09,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,2.0
4,-2.099202e+09,1.0,0.0,1.0,5.0,4.0,2.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520726,-1.429413e+09,6.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2,2.0
520727,-3.118370e+08,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0
520728,-3.141701e+08,7.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2,2.0
520729,1.090677e+09,1.0,1.0,1.0,9.0,9.0,8.0,1.0,2.0,1.0,0.0,0.0,9.0,0.0,0.0,2.0,1.0,1.0,0,0.0


In [295]:
sum(predictions==2)


412788

In [296]:
sum(Y_test==2)

387837

In [297]:
sum(predictions ==1)


1165

In [298]:
sum(Y_test==1)

7764

In [299]:
sum(predictions ==0)

102586

In [300]:
sum(Y_test==0)

107562

In [301]:
sum(predictions==3)

4192

In [302]:
sum(Y_test==3)

17568

In [307]:
#Purchase and Activate=0, Purchase and non-Activate=1, 
#non-Purchase and non-Activate=2, non-Purchase and Activate=3

dfpa = df2[df2["predicted cond"]==0]
dfpna = df2[df2["predicted cond"]==1]
dfnpna = df2[df2["predicted cond"]==2]
dfnpa = df2[df2["predicted cond"]==3]


In [306]:
dfpa.describe()

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
count,1.008050e+05,100805.000000,100805.000000,100805.000000,100805.000000,100805.00000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.000000,100805.0,100805.0
mean,1.655241e+06,1.954963,0.579475,0.837468,21.252418,8.01609,3.480829,1.247081,4.721641,0.968781,0.027032,0.034949,3.823739,0.471524,0.089619,1.171182,0.961609,0.933505,0.0,0.0
std,1.239824e+09,2.356356,0.514892,1.835604,30.382114,10.22192,3.935074,0.981824,6.524989,0.192431,0.183366,0.184728,6.596971,1.108072,0.288951,3.132793,0.203278,0.249146,0.0,0.0
min,-2.147464e+09,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,-1.075775e+09,0.000000,0.000000,0.000000,5.000000,2.00000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,0.0
50%,2.892278e+06,1.000000,1.000000,0.000000,12.000000,5.00000,2.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,0.0
75%,1.074154e+09,3.000000,1.000000,1.000000,26.000000,10.00000,4.000000,1.000000,6.000000,1.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.0
max,2.147446e+09,25.000000,6.000000,155.000000,1016.000000,203.00000,97.000000,41.000000,224.000000,3.000000,7.000000,3.000000,354.000000,10.000000,3.000000,117.000000,3.000000,1.000000,0.0,0.0


In [308]:
dfpna.describe()

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
count,6.810000e+02,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.0,681.000000,681.000000,681.0,681.000000,681.000000,681.000000,681.000000,681.000000,681.0,681.000000,681.0,681.0
mean,4.637913e+07,1.290749,0.139501,0.117474,1.450808,0.475771,0.346549,0.0,0.348018,0.242291,0.0,0.729809,0.659325,0.060206,0.016153,0.547724,0.0,0.286344,1.0,1.0
std,1.260967e+09,0.947467,0.346723,0.503812,3.668347,1.037425,0.694743,0.0,0.828287,0.428784,0.0,0.444385,2.590981,0.368885,0.126155,1.183650,0.0,0.452384,0.0,0.0
min,-2.131543e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0
25%,-1.069306e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0
50%,8.369522e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0
75%,1.154627e+09,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,1.000000,1.0,1.0
max,2.146733e+09,6.000000,1.000000,6.000000,31.000000,7.000000,4.000000,0.0,7.000000,1.000000,0.0,1.000000,34.000000,6.000000,1.000000,8.000000,0.0,1.000000,1.0,1.0


In [309]:
dfnpna.describe()

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
count,3.864980e+05,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.0,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.000000,386498.0,386498.000000,386498.0,386498.0
mean,3.870397e+05,3.837270,0.605819,0.724850,8.880108,2.263908,0.907917,0.0,1.586792,0.984422,0.021827,0.015320,3.396028,1.579757,0.087894,0.855161,0.0,0.000010,2.0,2.0
std,1.239820e+09,3.555049,0.506075,1.548869,20.517790,4.964731,2.033306,0.0,3.893198,0.124463,0.161140,0.122821,5.846036,2.060311,0.285788,2.626654,0.0,0.003217,0.0,0.0
min,-2.147475e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.0,2.0
25%,-1.073212e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.0,2.0
50%,-7.869715e+05,3.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.0,2.0
75%,1.073205e+09,7.000000,1.000000,1.000000,9.000000,2.000000,1.000000,0.0,2.000000,1.000000,0.000000,0.000000,6.000000,3.000000,0.000000,1.000000,0.0,0.000000,2.0,2.0
max,2.147479e+09,23.000000,8.000000,88.000000,1551.000000,199.000000,110.000000,0.0,358.000000,2.000000,7.000000,1.000000,189.000000,11.000000,4.000000,308.000000,0.0,1.000000,2.0,2.0


In [310]:
dfnpa.describe()

,account_id,1c,2c,3c,4c,5c,6c,8c,11c,12c,13c,15c,19c,21c,22c,24c,27c,28c,predicted cond,Y_test
count,2.320000e+03,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.0,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.000000,2320.0,2320.0
mean,-1.918262e+07,1.247845,0.478879,0.566379,17.175431,13.403017,5.881897,0.0,5.303017,1.018534,0.033190,0.003879,2.581897,0.379741,0.082759,0.624138,0.586207,0.294828,3.0,3.0
std,1.232109e+09,1.951823,0.533874,1.305076,34.012830,20.397052,9.117404,0.0,9.392683,0.158425,0.218234,0.062177,5.225837,1.137856,0.283292,1.618171,0.492618,0.456064,0.0,0.0
min,-2.146362e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,3.0
25%,-1.082454e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,3.0
50%,7.633958e+06,0.000000,0.000000,0.000000,5.000000,4.000000,2.000000,0.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.0,3.0
75%,1.003319e+09,2.000000,1.000000,1.000000,21.000000,20.000000,9.000000,0.0,7.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,3.0,3.0
max,2.147235e+09,18.000000,4.000000,29.000000,620.000000,264.000000,113.000000,0.0,156.000000,2.000000,4.000000,1.000000,59.000000,7.000000,2.000000,20.000000,1.000000,1.000000,3.0,3.0
